In [15]:
from datasets import get_dataset_names, get_all_datasets
from main_domain import parse_args, extend_args, check_args
from utils.conf import base_path, get_device
from backbone import get_backbone, get_backbone_class
from models import get_model

In [ ]:
import sys, shlex
cmd = "--dataset seq-cifar100 --model LEAR --lr 0.03 --batch_size 32 --n_epochs 2 --num_workers 0 --backbone lear"
sys.argv = ['main_domain.py'] + shlex.split(cmd)
args = parse_args()

[WARNING] 06-Jan-26 03:29:02 - Trying to load default configuration for model LEAR but no configuration file found in None.
[WARNING] 06-Jan-26 03:29:02 - Default configuration file not found for dataset seq-cifar100. Using the defaults specified in the dataset class (if available).
[INFO] 06-Jan-26 03:29:02 - `lr_scheduler` set to multisteplr, overrides default from dataset.
[ERROR] 06-Jan-26 03:29:02 - Could not retrieve git hash.


In [24]:
args

Namespace(dataset='seq-cifar100', model='LEAR', backbone='lear', load_best_args=False, dataset_config=None, model_config='default', num_classes=100, seed=None, permute_classes=False, base_path='./data/', results_path='results/', device=device(type='cuda', index=0), notes=None, eval_epochs=None, non_verbose=False, disable_log=False, num_workers=0, enable_other_metrics=False, debug_mode=False, inference_only=False, code_optimization=0, distributed='no', savecheck=None, save_checkpoint_mode='safe', loadcheck=None, ckpt_name=None, start_from=None, stop_after=None, wandb_name=None, wandb_entity=None, wandb_project=None, lr=0.03, batch_size=32, label_perc=1.0, label_perc_by_class=1.0, joint=0, eval_future=False, validation=None, validation_mode='current', fitting_mode='epochs', early_stopping_patience=5, early_stopping_metric='loss', early_stopping_freq=1, early_stopping_epsilon=1e-06, n_epochs=2, n_iters=None, optimizer='sgd', optim_wd=0.0, optim_mom=0.0, optim_nesterov=False, drop_last=Fal

In [21]:
args.fitting_mode

'epochs'

In [4]:
datasets = get_all_datasets(args)
datasets[0]

Loading dataset: seq-cifar100


In [5]:
extend_args(args, datasets[0])

[INFO] 06-Jan-26 03:29:28 - `wandb_entity` and `wandb_project` not set. Disabling wandb.


In [6]:
check_args(args, dataset=datasets[0])

[WARNING] 06-Jan-26 03:29:33 - Label noise is not available with multi-label datasets. If this is not multi-label, ignore this warning.


In [7]:
device = get_device(avail_devices=args.device)
args.device = device
args.device

[WARNING] 06-Jan-26 03:29:37 - Could not get memory from pynvml. Maybe try `pip install --force-reinstall gpustat`. module 'torch.cuda' has no attribute 'pynvml'
[INFO] 06-Jan-26 03:29:37 - Using device cuda:0


device(type='cuda', index=0)

In [25]:
args.inference_only

False

In [8]:
args.backbone

'lear'

In [9]:
backbone_class, backbone_args = get_backbone_class(args.backbone, return_args=True)
parsed_args = {arg: getattr(args, arg) for arg in backbone_args.keys()}
parsed_args

{'num_classes': 100}

In [ ]:
backbone = backbone_class(**parsed_args)

[INFO] 06-Jan-26 03:30:03 - Loading pretrained weights from Hugging Face hub (timm/vit_base_patch16_224.augreg2_in21k_ft_in1k)
[INFO] 06-Jan-26 03:30:04 - HTTP Request: HEAD https://huggingface.co/timm/vit_base_patch16_224.augreg2_in21k_ft_in1k/resolve/main/model.safetensors "HTTP/1.1 302 Found"
[INFO] 06-Jan-26 03:30:04 - [timm/vit_base_patch16_224.augreg2_in21k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
[INFO] 06-Jan-26 03:30:05 - Loading pretrained weights from Hugging Face hub (timm/vit_base_patch16_224.augreg2_in21k_ft_in1k)
[INFO] 06-Jan-26 03:30:06 - HTTP Request: HEAD https://huggingface.co/timm/vit_base_patch16_224.augreg2_in21k_ft_in1k/resolve/main/model.safetensors "HTTP/1.1 302 Found"
[INFO] 06-Jan-26 03:30:06 - [timm/vit_base_patch16_224.augreg2_in21k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


In [11]:
backbone

LEAR(
  (local_vitmodel): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none'

In [13]:
loss = datasets[0].get_loss()
loss

<function torch.nn.functional.cross_entropy(input: torch.Tensor, target: torch.Tensor, weight: Optional[torch.Tensor] = None, size_average: Optional[bool] = None, ignore_index: int = -100, reduce: Optional[bool] = None, reduction: str = 'mean', label_smoothing: float = 0.0) -> torch.Tensor>

In [14]:
datasets[0].get_transform()

Compose(
    ToPILImage()
    Compose(
    RandomCrop(size=(32, 32), padding=4)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=(0.5071, 0.4867, 0.4408), std=(0.2675, 0.2565, 0.2761))
)
)

In [17]:
model = get_model(args, backbone, loss, datasets[0].get_transform(), dataset=datasets[0])
model

Using LEAR as backbone


LEAR(
  (net): LEAR(
    (local_vitmodel): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=768, out_fea

In [28]:
model.N_CLASSES

100

In [32]:
model.net.forward

<bound method LEAR.forward of LEAR(
  (local_vitmodel): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          

In [34]:
model.net

LEAR(
  (local_vitmodel): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none'